In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Obsessive-Compulsive_Disorder/GSE60190'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genetic Neuropathology of Obsessive Psychiatric Syndromes"
!Series_summary	"Anorexia nervosa (AN), bulimia nervosa (BN), and obsessive-compulsive disorder (OCD) are complex psychiatric disorders with shared obsessive features, thought to arise from the interaction of multiple genes of small effect with environmental factors.  Potential candidate genes for AN, BN, and OCD have been identified through clinical association and neuroimaging studies; however, recent genome-wide association studies of eating disorders (ED) so far have failed to report significant findings. Additionally, few if any studies have interrogated postmortem brain tissue for evidence of eQTLs associated with candidate genes, which has particular promise as an approach to elucidating molecular mechanisms of association. We therefore selected single nucleotide polymorphisms (SNPs) based on candidate gene studies for AN, BN, and OCD from the literature, and examined the associatio

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # The dataset is based on the Illumina HumanHT-12 v3 microarray, which measures mRNA expression

# Determine the keys associated with the variables
trait_row = 1 if len(set([item.split(': ')[1] for item in {1: ['ocd: ED', 'ocd: Control', 'ocd: OCD']}[1]])) > 1 else None
age_row = 5 if len(set([item.split(': ')[1] for item in {5: ['age: 50.421917', 'age: 27.49863']}[5]])) > 1 else None
gender_row = 7 if len(set([item.split(': ')[1] for item in {7: ['Sex: F', 'Sex: M']}[7]])) > 1 else None

# Define the conversion functions
def convert_trait(value):
    value = value.split(': ')[1]
    if value == 'Control':
        return 0
    elif value in ['OCD', 'ED']:
        return 1
    return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(': ')[1]
    if value == 'F':
        return 0
    elif value == 'M':
        return 1
    return None

# Save cohort information
save_cohort_info('GSE60190', './preprocessed/Obsessive-Compulsive_Disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Obsessive-Compulsive_Disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Obsessive-Compulsive_Disorder/trait_data/GSE60190.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1467273': [1.0, 50.421917, 0.0], 'GSM1467274': [0.0, 27.49863, 1.0], 'GSM1467275': [1.0, 30.627397, 1.0], 'GSM1467276': [0.0, 61.167123, 1.0], 'GSM1467277': [0.0, 32.69589, 1.0], 'GSM1467278': [1.0, 39.213698, 0.0], 'GSM1467279': [0.0, 58.605479, 1.0], 'GSM1467280': [0.0, 49.2, 1.0], 'GSM1467281': [0.0, 41.041095, 1.0], 'GSM1467282': [0.0, 51.750684, 1.0], 'GSM1467283': [0.0, 50.89863, 1.0], 'GSM1467284': [0.0, 26.745205, 1.0], 'GSM1467285': [1.0, 29.104109, 1.0], 'GSM1467286': [1.0, 39.301369, 0.0], 'GSM1467287': [0.0, 48.978082, 1.0], 'GSM1467288': [1.0, 57.884931, 1.0], 'GSM1467289': [0.0, 28.364383, 1.0], 'GSM1467290': [0.0, 24.041095, 1.0], 'GSM1467291': [0.0, 19.268493, 0.0], 'GSM1467292': [1.0, 27.230136, 0.0], 'GSM1467293': [0.0, 46.605479, 1.0], 'GSM1467294': [0.0, 23.443835, 0.0], 'GSM1467295': [0.0, 51.038356, 1.0], 'GSM1467296': [0.0, 39.663013, 1.0], 'GSM1467297': [0.0, 46.109589, 1.0], 'GSM1467298': [0.0, 77.989041, 1.0], 'GSM1467299': [0.0, 46.967123, 1.0], 'GSM1467

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1725881', 'ILMN_1910180', 'ILMN_1804174', 'ILMN_1796063', 'ILMN_1811966'], 'nuID': ['rp13_p1x6D80lNLk3c', 'NEX0oqCV8.er4HVfU4', 'KyqQynMZxJcruyylEU', 'xXl7eXuF7sbPEp.KFI', '9ckqJrioiaej9_ajeQ'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['RefSeq', 'Unigene', 'RefSeq', 'RefSeq', 'RefSeq'], 'Search_Key': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'Transcript': ['ILMN_44919', 'ILMN_127219', 'ILMN_139282', 'ILMN_5006', 'ILMN_38756'], 'ILMN_Gene': ['LOC23117', 'HS.575038', 'FCGR2B', 'TRIM44', 'LOC653895'], 'Source_Reference_ID': ['XM_933824.1', 'Hs.575038', 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'RefSeq_ID': ['XM_933824.1', nan, 'XM_938851.1', 'NM_017583.3', 'XM_936379.1'], 'Unigene_ID': [nan, 'Hs.575038', nan, nan, nan], 'Entrez_Gene_ID': [23117.0, nan, 2213.0, 54765.0, 653895.0], 'GI': [89040007.0, 10437021.0, 88952550.0, 29029528.0, 89033487.0], 'Access

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for mapping identifiers to gene symbols.
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe.
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Obsessive-Compulsive_Disorder/gene_data/GSE60190.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Obsessive-Compulsive_Disorder')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE60190', './preprocessed/Obsessive-Compulsive_Disorder/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Obsessive-Compulsive_Disorder/GSE60190.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Obsessive-Compulsive_Disorder', the least common label is '1.0' with 31 occurrences. This represents 23.31% of the dataset.
The distribution of the feature 'Obsessive-Compulsive_Disorder' in this dataset is fine.

Quartiles for 'Age':
  25%: 30.043835
  50% (Median): 45.224657
  75%: 54.424657
Min: 16.183561
Max: 84.057534
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 36 occurrences. This represents 27.07% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

